# Energy Statistics

Simple notebook to extract general stats about the energy production and consumption.

In [2]:
import aioinflux
import os
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
kwargs = {
    'host': '192.168.178.47',
    'port': os.getenv('INFLUXDB_PORT'),
    'username': os.getenv('INFLUXDB_USER'),
    'password': os.getenv('INFLUXDB_PASS'),
    'db': os.getenv('INFLUXDB_DB'),
    'output': 'dataframe'
}

In [4]:
today = datetime.today().date()
print(f"Today is {today}")

Today is 2021-04-27


In [5]:
def round50(df):
    """Round values to 50s"""
    fac = 0.02
    return (df*fac).round(decimals=0)/fac

In [6]:
measurement = 'electricity_energy_watthour'
async with aioinflux.InfluxDBClient(**kwargs) as ifclient:
    df_energy = await ifclient.query(f"""
        SELECT * FROM {measurement}
    """)

In [7]:
# consider only last year
d = df_energy[today-pd.DateOffset(years=1):today]

round50(
    d
    .groupby(d.index.date)
    .sum()
    .quantile(q=[0.1, 0.25, 0.5, 0.75, 0.9])
)

/Users/christoph.sawade/Library/Python/3.8/lib/python/site-packages/pandas/core/indexes/base.py:5277: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)


,Consumption,FeedIn,Production,Purchased,SelfConsumption
0.10,5150.0,50.0,1000.0,2300.0,850.0
0.25,6050.0,1600.0,3450.0,2800.0,1650.0
0.50,7200.0,6850.0,9650.0,4000.0,2700.0
0.75,8400.0,12600.0,16550.0,6000.0,3850.0
0.90,9950.0,16300.0,20100.0,8600.0,4750.0


In [8]:
measurement = 'electricity_power_watt'
async with aioinflux.InfluxDBClient(**kwargs) as ifclient:
    df_power = await ifclient.query(f"""
        SELECT * FROM {measurement}
    """)

In [9]:
# consider only last year
d = df_power[today-pd.DateOffset(years=1):today]

# ignore lower half (quiescent current)
#d[d<=d.quantile(q=0.5, axis=0)]=np.nan

round50(
    d
    .quantile(q=[0.5, 0.75, 0.9, 0.95, 0.99], axis=0)
)

/Users/christoph.sawade/Library/Python/3.8/lib/python/site-packages/pandas/core/indexes/base.py:5277: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)


,Consumption,FeedIn,Production,Purchased,SelfConsumption
0.50,200.0,0.0,250.0,150.0,0.0
0.75,250.0,300.0,1100.0,250.0,150.0
0.90,350.0,1350.0,2050.0,300.0,250.0
0.95,950.0,1900.0,2450.0,500.0,300.0
0.99,2550.0,2350.0,2650.0,2000.0,1050.0
